#  Hello RabbitMQ!"

### 우선 기본적으로 연결해서 주고받는걸 해보겠습니다

우리가 해야 할 첫 번째 일은 패키지를 받고  RabbitMQ 서버와의 연결을 설정하는 것입니다.

In [1]:
! pip3 install pika

    100% |################################| 153kB 295kB/s ta 0:00:01


In [34]:
import pika

credentials = pika.PlainCredentials("user", "bitnami")
connection = pika.BlockingConnection(pika.ConnectionParameters('127.0.0.1',
                                                               '5672',
                                                               '/',
                                                               credentials=credentials))
channel = connection.channel()

## 보내기 - Sending

#### 우리의 첫 번째 프로그램은 하나의 메시지를 큐에 보냅니다.
#### 우선 큐를 지정해야 합니다.

In [35]:
channel.queue_declare(queue='hello')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=1', 'message_count=0', 'queue=hello'])>"])>

#### 이 시점에서 우리는 메시지를 보낼 준비가 되었습니다. <br> RabbitMQ에서는 메시지를 큐로 직접 보낼 수 없으며 항상 exchange을 거쳐야 합니다. <br> 하지만 일단은 빈 문자열로 exchange를 지정합니다. <br> 큐 이름은 routing_key 매개 변수에 지정해야합니다.

In [36]:
channel.basic_publish(exchange='xyz',
                      routing_key='hello',
                      body='Hello World!')
print(" [x] Sent 'Hello World!'")


 [x] Sent 'Hello World!'


http://localhost:15672 로 들어가서 확인해보기 

## 문제

### redis 실습에서 만들었던 가상 데이터를 이용해서 rabbitMQ로 전달하고 받는 쪽에서는 그걸 mongo DB에 넣는 것까지 진행 


### producer : 가상 데이터 퍼블리싱    /    consumer : mongo DB에 적재 

#### Hint 기존 redis에서 사용한 함수 
#### 조건 큐 이름은 mail_chan으로 하세요 

In [ ]:
def publish_sold_email_via_pubsub(conn, seller, item, price, buyer):
    data = {
        'seller_id': seller,
        'item_id': item,
        'price': price,
        'buyer_id': buyer,
        'time': time.time()
    }
    result = conn.publish('mail_channel',json.dumps(data))\n"
    print(result, data)

In [37]:
#### Hint2 기존 redis에서 사용한 루프

In [ ]:
import random

while True:
    publish_sold_email_via_pubsub(conn=conn,
                                  seller=random.choice(seller_list),
                                  item=random.choice(item_list),
                                  price=random.randrange(500,1000),
                                  buyer="buyer_{}".format(random.randrange(1000000)))
    time.sleep(random.randrange(1,5))

In [ ]:
credentials = pika.PlainCredentials("user", "bitnami")
connection = pika.BlockingConnection(pika.ConnectionParameters('127.0.0.1',
                                                               '5672',
                                                               '/',
                                                               credentials=credentials))
channel = connection.channel()

channel.queue_declare(queue='mail_chan')


def publish_sold_email_via_pubsub(conn, seller, item, price, buyer):
    data = {
        'seller_id': seller,
        'item_id': item,
        'price': price,
        'buyer_id': buyer,
        'time': time.time()
    }
    channel.basic_publish(exchange='xyz',
                      routing_key='mail_chan',
                      body='Hello World!')

In [ ]:
import random

while True:
    publish_sold_email_via_pubsub(conn=conn,
                                  seller=random.choice(seller_list),
                                  item=random.choice(item_list),
                                  price=random.randrange(500,1000),
                                  buyer="buyer_{}".format(random.randrange(1000000)))
    time.sleep(random.randrange(1,5))